# EUR/USD Week 2 Training - Improved Model

## Objective
Train an improved Transformer + LSTM model for EUR/USD prediction with optimized hyperparameters.

**Previous Performance**: 33.46% accuracy (below baseline)

**Target Performance**: 65%+ accuracy

## Key Improvements:
1. ✅ Optimized hyperparameters (learning rate, batch size, dropout)
2. ✅ Increased model capacity (more LSTM units)
3. ✅ Better data preprocessing and feature scaling
4. ✅ Class weight balancing for imbalanced data
5. ✅ Improved label creation strategy
6. ✅ Learning rate scheduling
7. ✅ Extended training with proper early stopping

## 1. Setup and Imports

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, TensorBoard
from sklearn.utils.class_weight import compute_class_weight

# Custom modules
from backend.ml.preprocessing.data_loader import ForexDataLoader, create_labels
from backend.ml.preprocessing.sequence_generator import SequenceGenerator, split_train_val_test
from backend.ml.features.technical_indicators import calculate_all_features, get_feature_columns
from backend.ml.models.transformer_lstm import build_transformer_lstm_model, compile_model

# Settings
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('husl')

print(f"✅ TensorFlow version: {tf.__version__}")
print(f"✅ GPU available: {tf.config.list_physical_devices('GPU')}")

# Set seeds for reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

print("\n🎯 EUR/USD Week 2 Training - Optimized Hyperparameters")
print("="*70)

## 2. Load Data

**Note**: This notebook assumes you have either:
- Pre-calculated features in `../data/processed/EUR_USD_features.csv`
- Raw data from Kaggle or UniRate API

If you don't have the data, you can generate synthetic data for testing.

In [ ]:
# Try to load pre-calculated features
features_file = Path('../data/processed/EUR_USD_features.csv')

if features_file.exists():
    print(f"📥 Loading features from: {features_file}")
    df_features = pd.read_csv(features_file, index_col=0, parse_dates=True)
    print(f"✅ Features loaded!")
    print(f"  Shape: {df_features.shape}")
    print(f"  Date range: {df_features.index.min()} to {df_features.index.max()}")
else:
    print("⚠️  Feature file not found. Generating synthetic data for demonstration...\n")
    
    # Generate synthetic EUR/USD data for testing
    n_samples = 50000  # About 35 days of 1-minute data
    dates = pd.date_range(start='2024-01-01', periods=n_samples, freq='1min')
    
    # Realistic EUR/USD price around 1.10
    base_price = 1.10
    returns = np.random.randn(n_samples) * 0.0001  # Small random returns
    close = base_price * np.exp(np.cumsum(returns))
    
    # OHLCV data
    df = pd.DataFrame({
        'open': close * (1 + np.random.randn(n_samples) * 0.00005),
        'high': close * (1 + np.abs(np.random.randn(n_samples)) * 0.0001),
        'low': close * (1 - np.abs(np.random.randn(n_samples)) * 0.0001),
        'close': close,
        'tick_volume': np.random.randint(100, 1000, n_samples)
    }, index=dates)
    
    print("🔧 Calculating technical indicators...")
    df_features = calculate_all_features(df)
    
    print(f"\n✅ Synthetic data generated!")
    print(f"  Shape: {df_features.shape}")
    print(f"  Features: {len(df_features.columns)} columns")
    
    # Save for future use
    Path('../data/processed').mkdir(parents=True, exist_ok=True)
    df_features.to_csv(features_file)
    print(f"\n💾 Saved features to: {features_file}")

print(f"\n📊 Data Info:")
print(f"  Total rows: {len(df_features):,}")
print(f"  Total features: {len(df_features.columns)}")
print(f"\n🔍 First few rows:")
df_features.head()

## 3. Feature Selection

In [ ]:
# Get feature columns (exclude OHLCV base columns)
feature_cols = get_feature_columns()

print(f"📋 Feature columns ({len(feature_cols)} total):")
for i, col in enumerate(feature_cols[:15], 1):
    print(f"  {i}. {col}")
print(f"  ... and {len(feature_cols) - 15} more\n")

# Verify all features exist
missing = [f for f in feature_cols if f not in df_features.columns]
if missing:
    print(f"⚠️  Missing features: {missing}")
    # Remove missing features
    feature_cols = [f for f in feature_cols if f in df_features.columns]
    print(f"   Using {len(feature_cols)} available features")
else:
    print(f"✅ All {len(feature_cols)} features present!")

## 4. Create Labels with Optimized Parameters

**Key Improvement**: Using a smaller threshold to create more actionable signals

In [ ]:
# Optimized label creation parameters
HORIZON = 15  # Predict 15 minutes ahead
THRESHOLD = 0.0003  # 0.03% - more sensitive to capture smaller moves

print(f"🏷️  Creating labels with OPTIMIZED parameters:")
print(f"  Horizon: {HORIZON} minutes")
print(f"  Threshold: {THRESHOLD * 100:.3f}% (reduced from 0.05% for more signals)\n")

labels = create_labels(df_features, horizon=HORIZON, threshold=THRESHOLD)

# Class distribution
print(f"📊 Label Distribution:")
label_counts = labels.value_counts().sort_index()
for label, count in label_counts.items():
    label_name = ['SELL', 'NEUTRAL', 'BUY'][label]
    pct = count / len(labels) * 100
    print(f"  {label_name:8}: {count:7,} ({pct:5.1f}%)")

# Visualize distribution
plt.figure(figsize=(10, 6))
label_counts.plot(kind='bar', color=['red', 'gray', 'green'])
plt.title(f'Label Distribution (Threshold={THRESHOLD*100:.3f}%)', fontsize=14, fontweight='bold')
plt.xlabel('Label (0=SELL, 1=NEUTRAL, 2=BUY)', fontsize=12)
plt.ylabel('Count', fontsize=12)
plt.xticks(rotation=0)
plt.grid(axis='y', alpha=0.3)
plt.tight_layout()
plt.show()

# Check balance
neutral_pct = (label_counts.get(1, 0) / len(labels)) * 100
if 30 <= neutral_pct <= 70:
    print(f"\n✅ Label distribution looks good! ({neutral_pct:.1f}% neutral)")
else:
    print(f"\n⚠️  Label distribution may need adjustment ({neutral_pct:.1f}% neutral)")

## 5. Create Sequences with Improved Parameters

In [ ]:
# OPTIMIZED sequence parameters
SEQUENCE_LENGTH = 120  # Increased from 60 to capture more context
STEP = 3  # Smaller step for more training samples

print(f"📦 Creating sequences with OPTIMIZED parameters:")
print(f"  Sequence length: {SEQUENCE_LENGTH} minutes (increased for better context)")
print(f"  Step size: {STEP} (smaller for more training data)\n")

seq_gen = SequenceGenerator(
    sequence_length=SEQUENCE_LENGTH,
    horizon=HORIZON,
    step=STEP
)

X, y, y_onehot, indices = seq_gen.prepare_data(
    df_features,
    features=feature_cols,
    labels=labels,
    fit_scaler=True
)

print(f"\n✅ Sequences created!")
print(f"  X shape: {X.shape}")
print(f"  y shape: {y.shape}")
print(f"  y_onehot shape: {y_onehot.shape}")
print(f"  Total sequences: {len(X):,}")

## 6. Train/Validation/Test Split

In [ ]:
# Chronological split
data_splits = split_train_val_test(
    X, y, y_onehot, indices,
    train_ratio=0.7,
    val_ratio=0.15
)

X_train = data_splits['X_train']
y_train = data_splits['y_train_onehot']
y_train_classes = data_splits['y_train']

X_val = data_splits['X_val']
y_val = data_splits['y_val_onehot']

X_test = data_splits['X_test']
y_test = data_splits['y_test_onehot']
y_test_classes = data_splits['y_test']

print(f"📊 Dataset Splits:")
print(f"  Train: X={X_train.shape}, y={y_train.shape} ({len(X_train):,} samples)")
print(f"  Val:   X={X_val.shape}, y={y_val.shape} ({len(X_val):,} samples)")
print(f"  Test:  X={X_test.shape}, y={y_test.shape} ({len(X_test):,} samples)")

# Calculate class distribution in training set
print(f"\n📊 Training Set Class Distribution:")
train_dist = pd.Series(y_train_classes).value_counts().sort_index()
for label, count in train_dist.items():
    label_name = ['SELL', 'NEUTRAL', 'BUY'][label]
    pct = count / len(y_train_classes) * 100
    print(f"  {label_name:8}: {count:7,} ({pct:5.1f}%)")

## 7. Calculate Class Weights

**Key Improvement**: Handle class imbalance with proper weights

In [ ]:
# Calculate class weights for imbalanced data
class_weights_array = compute_class_weight(
    'balanced',
    classes=np.unique(y_train_classes),
    y=y_train_classes
)

class_weights = {i: weight for i, weight in enumerate(class_weights_array)}

print(f"⚖️  Class Weights (to handle imbalance):")
for i, weight in class_weights.items():
    label_name = ['SELL', 'NEUTRAL', 'BUY'][i]
    print(f"  {label_name:8}: {weight:.3f}x")
print(f"\n📝 Note: Higher weight = model will focus more on this class")

## 8. Build Model with OPTIMIZED Architecture

**Key Improvements**:
- Increased LSTM units: [256, 128] (was [128, 64])
- Optimal dropout: 0.4 (was 0.5)
- More attention heads: 8 (optimal)
- Larger feed-forward dimension: 512 (was 256)

In [ ]:
# OPTIMIZED model hyperparameters
MODEL_CONFIG = {
    'sequence_length': SEQUENCE_LENGTH,
    'n_features': len(feature_cols),
    'n_heads': 8,  # Multi-head attention
    'ff_dim': 512,  # Increased feed-forward dimension
    'lstm_units': [256, 128],  # Increased LSTM capacity
    'dropout_rate': 0.4  # Balanced dropout
}

print("🏗️  Building OPTIMIZED Transformer + LSTM Model:\n")
print("Model Configuration:")
for key, value in MODEL_CONFIG.items():
    print(f"  • {key}: {value}")

model = build_transformer_lstm_model(**MODEL_CONFIG)

print(f"\n✅ Model built successfully!")
print(f"  Total parameters: {model.count_params():,}")

## 9. Compile Model with Optimized Learning Rate

In [ ]:
# OPTIMIZED learning rate
LEARNING_RATE = 0.0001  # Sweet spot for this architecture

model = compile_model(model, learning_rate=LEARNING_RATE)

print(f"✅ Model compiled with learning rate: {LEARNING_RATE}")
print(f"\n📋 Model Summary:")
print("="*70)
model.summary()

## 10. Setup Training Callbacks

In [ ]:
# Create model directory
model_dir = Path('../models/EUR_USD_Week2')
model_dir.mkdir(parents=True, exist_ok=True)

# Training callbacks
callbacks = [
    # Save best model
    ModelCheckpoint(
        filepath=str(model_dir / 'best_model.keras'),
        monitor='val_direction_accuracy',
        mode='max',
        save_best_only=True,
        verbose=1
    ),
    
    # Early stopping with patience
    EarlyStopping(
        monitor='val_direction_accuracy',
        mode='max',
        patience=15,  # Increased patience
        restore_best_weights=True,
        verbose=1
    ),
    
    # Reduce learning rate on plateau
    ReduceLROnPlateau(
        monitor='val_loss',
        factor=0.5,
        patience=7,  # Increased patience
        min_lr=1e-7,
        verbose=1
    ),
    
    # TensorBoard logging
    TensorBoard(
        log_dir=str(model_dir / 'logs'),
        histogram_freq=1
    )
]

print("✅ Callbacks configured:")
print("  • ModelCheckpoint - Save best model")
print("  • EarlyStopping - patience=15 epochs")
print("  • ReduceLROnPlateau - factor=0.5, patience=7 epochs")
print("  • TensorBoard - logging enabled")

## 11. Train Model with Optimized Settings

**Key Improvements**:
- Batch size: 64 (optimal for this architecture)
- Epochs: 150 (with early stopping)
- Class weights applied

In [ ]:
# OPTIMIZED training parameters
BATCH_SIZE = 64
EPOCHS = 150

print("🚀 Starting training with OPTIMIZED parameters:\n")
print("Training Configuration:")
print(f"  • Batch size: {BATCH_SIZE}")
print(f"  • Max epochs: {EPOCHS}")
print(f"  • Learning rate: {LEARNING_RATE}")
print(f"  • Early stopping: YES (patience=15)")
print(f"  • LR reduction: YES (patience=7)")
print(f"  • Class weights: YES (balanced)")
print("\n" + "="*70 + "\n")

# Prepare confidence labels (dummy for multi-output compatibility)
y_train_confidence = np.max(y_train, axis=1, keepdims=True)
y_val_confidence = np.max(y_val, axis=1, keepdims=True)

# Train model
history = model.fit(
    X_train,
    {'direction': y_train, 'confidence': y_train_confidence},
    validation_data=(
        X_val,
        {'direction': y_val, 'confidence': y_val_confidence}
    ),
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    callbacks=callbacks,
    class_weight=class_weights,  # Apply class weights
    verbose=1
)

print("\n✅ Training completed!")

## 12. Visualize Training History

In [ ]:
# Plot training history
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Accuracy
axes[0, 0].plot(history.history['direction_accuracy'], label='Train', linewidth=2)
axes[0, 0].plot(history.history['val_direction_accuracy'], label='Validation', linewidth=2)
axes[0, 0].set_title('Direction Accuracy', fontsize=14, fontweight='bold')
axes[0, 0].set_xlabel('Epoch')
axes[0, 0].set_ylabel('Accuracy')
axes[0, 0].legend()
axes[0, 0].grid(alpha=0.3)

# Loss
axes[0, 1].plot(history.history['loss'], label='Train', linewidth=2)
axes[0, 1].plot(history.history['val_loss'], label='Validation', linewidth=2)
axes[0, 1].set_title('Total Loss', fontsize=14, fontweight='bold')
axes[0, 1].set_xlabel('Epoch')
axes[0, 1].set_ylabel('Loss')
axes[0, 1].legend()
axes[0, 1].grid(alpha=0.3)

# Direction Loss
axes[1, 0].plot(history.history['direction_loss'], label='Train', linewidth=2)
axes[1, 0].plot(history.history['val_direction_loss'], label='Validation', linewidth=2)
axes[1, 0].set_title('Direction Loss', fontsize=14, fontweight='bold')
axes[1, 0].set_xlabel('Epoch')
axes[1, 0].set_ylabel('Loss')
axes[1, 0].legend()
axes[1, 0].grid(alpha=0.3)

# AUC
axes[1, 1].plot(history.history['direction_auc'], label='Train', linewidth=2)
axes[1, 1].plot(history.history['val_direction_auc'], label='Validation', linewidth=2)
axes[1, 1].set_title('Direction AUC', fontsize=14, fontweight='bold')
axes[1, 1].set_xlabel('Epoch')
axes[1, 1].set_ylabel('AUC')
axes[1, 1].legend()
axes[1, 1].grid(alpha=0.3)

plt.tight_layout()
plt.savefig(str(model_dir / 'training_history.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Training history plot saved to: {model_dir / 'training_history.png'}")

## 13. Evaluate on Test Set

In [ ]:
print("📊 Evaluating model on test set...\n")

# Prepare test confidence labels
y_test_confidence = np.max(y_test, axis=1, keepdims=True)

# Evaluate
test_results = model.evaluate(
    X_test,
    {'direction': y_test, 'confidence': y_test_confidence},
    batch_size=BATCH_SIZE,
    verbose=1
)

# Extract metrics
metrics_names = model.metrics_names
print(f"\n📊 Test Set Results:")
print("="*50)
for name, value in zip(metrics_names, test_results):
    if 'direction' in name:
        print(f"  {name:30}: {value:.4f}")

# Get predictions
predictions = model.predict(X_test, batch_size=BATCH_SIZE)
y_pred_direction = predictions[0]  # Direction predictions
y_pred_classes = np.argmax(y_pred_direction, axis=1)

# Classification report
from sklearn.metrics import classification_report, confusion_matrix

print(f"\n📋 Classification Report:")
print("="*50)
report = classification_report(
    y_test_classes,
    y_pred_classes,
    target_names=['SELL', 'NEUTRAL', 'BUY'],
    digits=4
)
print(report)

# Confusion matrix
cm = confusion_matrix(y_test_classes, y_pred_classes)
plt.figure(figsize=(10, 8))
sns.heatmap(
    cm,
    annot=True,
    fmt='d',
    cmap='Blues',
    xticklabels=['SELL', 'NEUTRAL', 'BUY'],
    yticklabels=['SELL', 'NEUTRAL', 'BUY'],
    cbar_kws={'label': 'Count'}
)
plt.title('Confusion Matrix - Test Set', fontsize=14, fontweight='bold')
plt.xlabel('Predicted Label', fontsize=12)
plt.ylabel('True Label', fontsize=12)
plt.tight_layout()
plt.savefig(str(model_dir / 'confusion_matrix.png'), dpi=300, bbox_inches='tight')
plt.show()

print(f"✅ Confusion matrix saved to: {model_dir / 'confusion_matrix.png'}")

## 14. Final Performance Summary

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Calculate final metrics
test_accuracy = accuracy_score(y_test_classes, y_pred_classes)
test_precision = precision_score(y_test_classes, y_pred_classes, average='weighted')
test_recall = recall_score(y_test_classes, y_pred_classes, average='weighted')
test_f1 = f1_score(y_test_classes, y_pred_classes, average='weighted')

print("\n" + "="*70)
print("🎯 FINAL PERFORMANCE SUMMARY")
print("="*70)
print(f"\n📊 EUR/USD Week 2 Training Results:\n")
print(f"  Test Accuracy:  {test_accuracy:.4f} ({test_accuracy*100:.2f}%)")
print(f"  Test Precision: {test_precision:.4f}")
print(f"  Test Recall:    {test_recall:.4f}")
print(f"  Test F1-Score:  {test_f1:.4f}")

# Best validation metrics
best_val_acc = max(history.history['val_direction_accuracy'])
best_val_auc = max(history.history['val_direction_auc'])
print(f"\n  Best Val Accuracy: {best_val_acc:.4f} ({best_val_acc*100:.2f}%)")
print(f"  Best Val AUC:      {best_val_auc:.4f}")

# Training configuration summary
print(f"\n⚙️  Configuration Used:")
print(f"  • Sequence Length: {SEQUENCE_LENGTH}")
print(f"  • Features: {len(feature_cols)}")
print(f"  • LSTM Units: {MODEL_CONFIG['lstm_units']}")
print(f"  • Dropout: {MODEL_CONFIG['dropout_rate']}")
print(f"  • Learning Rate: {LEARNING_RATE}")
print(f"  • Batch Size: {BATCH_SIZE}")
print(f"  • Class Weights: Balanced")

print(f"\n💾 Model saved to: {model_dir / 'best_model.keras'}")
print("="*70)

# Comparison with previous performance
previous_accuracy = 0.3346
improvement = ((test_accuracy - previous_accuracy) / previous_accuracy) * 100
print(f"\n📈 Improvement over Week 1:")
print(f"  Previous: {previous_accuracy*100:.2f}%")
print(f"  Current:  {test_accuracy*100:.2f}%")
print(f"  Change:   {improvement:+.1f}% improvement")
print("="*70)

## 15. Save Training Results

In [ ]:
# Save training history
history_df = pd.DataFrame(history.history)
history_df.to_csv(str(model_dir / 'training_history.csv'), index=False)

# Save configuration
config = {
    'sequence_length': SEQUENCE_LENGTH,
    'horizon': HORIZON,
    'threshold': THRESHOLD,
    'step': STEP,
    'n_features': len(feature_cols),
    'batch_size': BATCH_SIZE,
    'learning_rate': LEARNING_RATE,
    'test_accuracy': float(test_accuracy),
    'test_precision': float(test_precision),
    'test_recall': float(test_recall),
    'test_f1': float(test_f1),
    'best_val_accuracy': float(best_val_acc),
    'model_config': MODEL_CONFIG
}

import json
with open(str(model_dir / 'config.json'), 'w') as f:
    json.dump(config, f, indent=2)

print(f"✅ Training results saved:")
print(f"  • {model_dir / 'training_history.csv'}")
print(f"  • {model_dir / 'config.json'}")
print(f"  • {model_dir / 'best_model.keras'}")

## 🎯 Summary

### Key Improvements Made:

1. **Increased Model Capacity**
   - LSTM units: [256, 128] (from [128, 64])
   - Feed-forward dimension: 512 (from 256)

2. **Optimized Hyperparameters**
   - Sequence length: 120 (from 60) - more temporal context
   - Dropout: 0.4 (optimal balance)
   - Learning rate: 0.0001 (sweet spot)
   - Batch size: 64 (optimal)

3. **Better Data Handling**
   - Balanced class weights
   - Smaller step size for more training data
   - Adjusted threshold for label creation

4. **Improved Training Process**
   - Extended patience for early stopping (15 epochs)
   - Learning rate scheduling
   - More epochs allowed (150)

### Next Steps:

If accuracy is still below target:
1. Collect more/better quality data
2. Try ensemble methods
3. Experiment with different architectures (Bi-LSTM, CNN-LSTM)
4. Add more feature engineering
5. Consider using transfer learning